# **Project 2:**

In [1]:
import numpy as np
import pandas as pd
from src.Ward import Ward, initialize_wards
from src.Patient import Patient, initialize_patients

## initialization

In [2]:
F_bed_capacity = 30 ##### Number to optimize #####

#patient type from 'A' to 'F'
patient_types = ['A', 'B', 'C', 'D', 'E', 'F']
bed_capacity = np.array([55,40,30,20,20, F_bed_capacity])
arrivals_pr_day = np.array([14.5,11.0,8.0,6.5,5.0, 13.0])
mean_length_of_stay = np.array([2.9,4.0,4.5,1.4,3.9, 2.0])
urgency_points = np.array([7,5,2,10,5, 0])

#reloaction probability
relocation_probability = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
    ])

#Dataframe containing all the data (indexed by patient type)
df = pd.DataFrame({
    'Patient Type': patient_types,
    'Bed Capacity': bed_capacity,
    'Arrivals per day': arrivals_pr_day,
    'Mean Length of Stay': mean_length_of_stay,
    'Urgency Points': urgency_points
    }).set_index('Patient Type')
df

,Bed Capacity,Arrivals per day,Mean Length of Stay,Urgency Points
Patient Type,,,,
A,55,14.5,2.9,7
B,40,11.0,4.0,5
C,30,8.0,4.5,2
D,20,6.5,1.4,10
E,20,5.0,3.9,5
F,30,13.0,2.0,0


In [3]:
wards = initialize_wards(df) #list of wards
wards

[A Ward with 55 beds and 0 patients currently admitted.,
 B Ward with 40 beds and 0 patients currently admitted.,
 C Ward with 30 beds and 0 patients currently admitted.,
 D Ward with 20 beds and 0 patients currently admitted.,
 E Ward with 20 beds and 0 patients currently admitted.,
 F Ward with 30 beds and 0 patients currently admitted.]

In [4]:
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])

In [5]:
total_time = 100 #total days to permit admissions
patients = initialize_patients(total_time, patient_types, arrival_interval_function, occupancy_time_function)
patients

[Patient A with Admission at time 0.0287794062975207,
 Patient A with Admission at time 0.030490544466580876,
 Patient B with Admission at time 0.034585144876150976,
 Patient A with Admission at time 0.03537836854095135,
 Patient B with Admission at time 0.038616551207712284,
 Patient C with Admission at time 0.043157698369711534,
 Patient F with Admission at time 0.04481714146102803,
 Patient A with Admission at time 0.0481203691172832,
 Patient A with Admission at time 0.05262803911342964,
 Patient D with Admission at time 0.105974946424535,
 Patient B with Admission at time 0.10702969526861641,
 Patient A with Admission at time 0.11077254500405015,
 Patient C with Admission at time 0.12373868907739603,
 Patient F with Admission at time 0.12416932748551515,
 Patient D with Admission at time 0.14074277054709042,
 Patient E with Admission at time 0.14644836296360011,
 Patient C with Admission at time 0.1477230428553006,
 Patient D with Admission at time 0.17685551011016667,
 Patient A 

In [6]:
def simulation_loop(patients, wards, relocation_probabilities):
    """Runs the simulation loop

    Args:
        patients (list): list of patient objects
        wards (list): list of ward objects
        relocation_probabilities (np.array): probability matrix for patient relocation

    Returns:
        urgency_points, total_lost: total urgency points, and total lost during the simulation
    """
    urgency_points = 0
    total_lost = 0
    while len(patients) > 0:
        patient = patients.pop(0)
        print(patient)
        if patient.next_event == "Admission":
            urgency_points += patient.occupy_bed(wards, patients, relocation_probabilities)
            total_lost += patient.lost
        else:
            patient.discharge(wards)
    return urgency_points, total_lost

In [7]:
urgency_points, total_lost = simulation_loop(patients, wards, relocation_probability)

Patient A with Admission at time 0.0287794062975207
Patient A with Admission at time 0.030490544466580876
Patient B with Admission at time 0.034585144876150976
Patient A with Admission at time 0.03537836854095135
Patient B with Admission at time 0.038616551207712284
Patient C with Admission at time 0.043157698369711534
Patient F with Admission at time 0.04481714146102803
Patient A with Admission at time 0.0481203691172832
Patient A with Admission at time 0.05262803911342964
Patient D with Admission at time 0.105974946424535
Patient B with Admission at time 0.10702969526861641
Patient A with Admission at time 0.11077254500405015
Patient C with Admission at time 0.12373868907739603
Patient F with Admission at time 0.12416932748551515
Patient D with Admission at time 0.14074277054709042
Patient E with Admission at time 0.14644836296360011
Patient C with Admission at time 0.1477230428553006
Patient D with Admission at time 0.17685551011016667
Patient A with Admission at time 0.198449894731

In [10]:
print("Urgency points: ", urgency_points)
print("Total lost: ", total_lost)

Urgency points:  3051
Total lost:  221
